In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare(engine, reflect = True)

In [6]:
# View all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = base.classes.measurement
Station = base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [9]:
# Display the Measurement row's columns and data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x22237108308>,
 'id': 1,
 'date': '2010-01-01',
 'tobs': 65.0,
 'prcp': 0.08,
 'station': 'USC00519397'}

In [10]:
# Display the Station row's columns and data in dictionary format
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x22237114108>,
 'station': 'USC00519397',
 'latitude': 21.2716,
 'id': 1,
 'elevation': 3.0,
 'name': 'WAIKIKI 717.2, HI US',
 'longitude': -157.8168}

# Exploratory Precipitation Analysis

In [11]:
# Find the most recent date in the data set.
most_recent = session.query(Measurement).order_by(Measurement.date.desc()).first()
newest_date = most_recent.date
print(newest_date)

2017-08-23


In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
from datetime import datetime
from dateutil.relativedelta import relativedelta
dt_new_date = datetime.strptime(newest_date, '%Y-%m-%d').date()

one_year_ago = dt_new_date - relativedelta(years = 1)

# Perform a query to retrieve the data and precipitation scores
from sqlalchemy import func
latest_year = session.query(Measurement.station,Measurement.prcp,Measurement.date,Measurement.tobs).\
filter(Measurement.date>=one_year_ago).all()

In [17]:
latest_year[1]

('USC00519397', 0.08, '2016-08-24', 79.0)

In [14]:
conn= engine.connect()

In [15]:
import pandas as pd

In [25]:
# Save the query results as a Pandas DataFrame and set the index to the date column
latest_df = pd.DataFrame.from_records(latest_year,
                                     index = 'date',
                                     columns = ["station",
                                               "prcp",
                                               "date",
                                               "tobs"])
latest_df.head()
# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data

,station,prcp,tobs
date,,,
2016-08-23,USC00519397,0.00,81.0
2016-08-24,USC00519397,0.08,79.0
2016-08-25,USC00519397,0.08,80.0
2016-08-26,USC00519397,0.00,79.0
2016-08-27,USC00519397,0.00,77.0


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()